# Script for a period

In [ ]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [3]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [5]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [8]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [ ]:
#unique_change_times = df['changeTime'].unique()
#print(unique_change_times)

['2025-02-25T09:56:44.373405+03:00' '2025-02-25T09:56:36.977327+03:00'
 '2025-02-10T15:50:50.654674+03:00' '2025-02-10T15:50:46.711076+03:00'
 '2024-12-28T14:56:51.438476+03:00' '2025-02-10T15:50:45.190525+03:00'
 '2025-02-06T14:45:40.972398+03:00' '2024-12-28T14:59:03.412003+03:00'
 '2025-02-10T15:50:43.275656+03:00' '2025-02-25T10:07:05.519407+03:00'
 '2025-01-06T19:56:55.030411+03:00' '2025-01-31T16:14:36.03518+03:00'
 '2025-02-26T15:24:26.898332+03:00' '2025-01-24T17:28:07.565413+03:00'
 '2025-01-24T17:27:45.032317+03:00' '2025-03-03T10:51:20.471041+03:00'
 '2023-06-09T17:13:45.296789+03:00' '2023-07-17T17:56:44.162491+03:00'
 '2023-09-11T19:47:52.142401+03:00' '2023-09-13T19:02:34.535107+03:00'
 '2023-09-11T17:14:54.50553+03:00' '2023-09-13T14:53:09.428335+03:00'
 '2023-12-11T23:45:14.286592+03:00' '2023-12-11T22:31:06.941346+03:00'
 '2024-07-15T12:16:26.499587+03:00' '2024-04-11T10:53:48.400039+03:00'
 '2024-05-13T10:13:17.846636+03:00' '2024-05-20T16:18:30.678177+03:00'
 '2024-0

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [9]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# List to store all campaign data
all_campaign_data_guten = []
all_campaign_data_giper = []
all_campaign_data_kitchen = []
all_campaign_data_smart = []

# Iterate over each chunk and send a POST request
for idx, (chunk_guten, chunk_giper, chunk_kitchen,chunk_smart) in enumerate(zip(campaign_chunks_guten, campaign_chunks_giper, campaign_chunks_kitchen,campaign_chunks_smart)):
    
    # Send the POST request
    response_guten = requests.post(url, params=query_params, json=chunk_guten, headers=headers_guten)
    time.sleep(1)
    response_giper = requests.post(url, params=query_params, json=chunk_giper, headers=headers_giper)
    time.sleep(1)
    response_kitchen = requests.post(url, params=query_params, json=chunk_kitchen, headers=headers_kitchen)
    time.sleep(1)
    response_smart = requests.post(url, params=query_params, json=chunk_smart, headers=headers_smart)
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
        # Parse the JSON response
        data_guten = response_guten.json()
        data_giper = response_giper.json()
        data_kitchen = response_kitchen.json()
        data_smart = response_smart.json()
        all_campaign_data_guten.extend(data_guten)
        all_campaign_data_giper.extend(data_giper)
        all_campaign_data_kitchen.extend(data_kitchen)
        all_campaign_data_smart.extend(data_smart)
        print("Data retrieved successfully")
        #print(f"Response for Chunk {idx + 1}: {data_guten}")
    else:
        print(f"Error for Chunk {idx + 1}: {response_guten.status_code,response_giper.status_code,response_kitchen.status_code,response_smart.status_code}, {response_guten.text,response_giper.text,response_kitchen.text,response_smart.text}")
        
        
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully
Combined Campaign Data


In [10]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2024-11-27,2024-10-31,2024-10-31,"Electrolux_Конвекторы 2000,1500_31.10.2024",21040635,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
1,2024-11-27,2024-10-31,2024-10-31,Electolux_Конвекторы 1000_31.10.2024,21040485,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
2,2024-11-27,2024-10-31,2024-11-22,Модули Ballu_31.10.2024,21040251,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
3,2024-12-09,2024-10-04,2024-11-15,ELIKOR_Врезные блоки_2 SKU_04.10.2024,20397195,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
4,2024-12-09,2024-10-03,2024-11-07,DEERMA_Увлажнители_03.10.2024,20354793,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
101,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
102,2100-01-01,2024-03-21,2025-02-20,Электробритвы Braun_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
103,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
104,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [ ]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Automatically get yesterday's date
yesterday = date.today().replace(day=date.today().day - 1)

specific_date = str(yesterday)  # Replace with your desired date

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_guten, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_guten.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

### Giper

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_giper, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_giper.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

# Save the combined data to a text file
#with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    #json.dump(all_campaign_data_giper, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
#print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


### Kitchen

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_kitchen, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_kitchen.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

Data retrived successfully
Combined Campaign Data


### Smart

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_smart, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_smart.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_smart_df = pd.json_normalize(all_campaign_data_smart)

Data retrived successfully
Combined Campaign Data


In [ ]:
campaign_smart_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,90217,4590,5.09,1.32,6061.44,379,63,1.37,67,373238,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 235350...",19597465,2024-11-01,2024-11-30
1,173369,10257,5.92,1.06,10876.90,904,132,1.29,142,1177551,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 235490...",20822563,2024-11-01,2024-11-30
2,31763,1866,5.87,1.27,2362.31,123,20,1.07,20,306374,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 206314...",20822524,2024-11-01,2024-11-30
3,2260,72,3.19,5.49,395.42,3,0,0.00,0,0,"[{'date': '2024-11-28T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 288465...",21693448,2024-11-28,2024-11-30
4,4454,123,2.76,6.75,830.00,8,2,1.63,2,2641,"[{'date': '2024-11-28T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 288465...",21693077,2024-11-28,2024-11-30
5,166823,8718,5.23,1.29,11249.94,614,77,0.88,78,760023,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 235338...",15562363,2024-11-01,2024-11-30


In [ ]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2024-11-01 03:00:00,262432608,Модуль конвектора Evolution Transformer BEC-EV...,17,0,0.00,0.00,1.12,0,0,0.0,0,0,21040251
1,2024-11-01 03:00:00,262432606,Модуль конвектора Evolution Transformer BEC-EV...,9,0,0.00,0.00,0.90,0,0,0.0,0,0,21040251
2,2024-11-01 03:00:00,262432607,Модуль конвектора Evolution Transformer BEC-EV...,16,0,0.00,0.00,1.70,0,0,0.0,0,0,21040251
3,2024-11-01 03:00:00,262432606,Модуль конвектора Evolution Transformer BEC-EV...,35,0,0.00,0.00,3.47,0,0,0.0,0,0,21040251
4,2024-11-01 03:00:00,262432608,Модуль конвектора Evolution Transformer BEC-EV...,62,2,3.23,2.08,4.16,0,0,0.0,0,0,21040251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,2024-11-19 03:00:00,257314011,Пылесос вертикальный компактный DX700 Pro,91,2,2.20,6.67,13.34,0,0,0.0,0,0,19994021
1298,2024-11-19 03:00:00,257314011,Пылесос вертикальный компактный DX700 Pro,26,0,0.00,0.00,5.39,0,0,0.0,0,0,19994021
1299,2024-11-20 03:00:00,257314011,Пылесос вертикальный компактный DX700 Pro,1,0,0.00,0.00,0.07,0,0,0.0,0,0,19994021
1300,2024-11-20 03:00:00,257314011,Пылесос вертикальный компактный DX700 Pro,1,0,0.00,0.00,0.07,0,0,0.0,0,0,19994021


In [ ]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,123116122,2024-11-01 03:00:00,Эпилятор женский электрический 3в1 CR-8803,2313,72,3.470000,1.545000,120.74,0,0,0.000000,0,0,20139599,WB-GutenTech,Wildberries
1,135577637,2024-11-01 03:00:00,Сменные кассеты для бритья Fusion 5 ProGlide 2 шт,2373,68,2.743333,3.510000,222.85,0,1,0.723333,1,1488,19263710,WB-GutenTech,Wildberries
2,137609384,2024-11-01 03:00:00,Бритва станок Fusion5 ProGlide Power Flexball ...,1844,72,3.593333,3.640000,172.25,0,1,1.450000,1,2259,19263710,WB-GutenTech,Wildberries
3,137609389,2024-11-01 03:00:00,Бритва станок Fusion 5 ProGlide с 3 кассетами,1618,38,2.570000,4.306667,152.17,0,1,11.110000,1,2043,19263710,WB-GutenTech,Wildberries
4,149281061,2024-11-01 03:00:00,Кусторез аккумуляторный садовый CSH 360,173,0,0.000000,0.000000,16.08,0,0,0.000000,0,0,20175678,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,288465478,2024-11-30 03:00:00,Набор Breeze бритва с кассетой и шампунь 250мл,204,3,1.070000,11.620000,50.51,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
2383,288465480,2024-11-30 03:00:00,Набор Gillette Venus с гелем для бритья,340,10,3.246667,7.906667,76.96,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
2384,288465481,2024-11-30 03:00:00,Набор Gillette Venus с чехлом для бритвы,351,15,3.223333,3.100000,77.30,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
2385,288465484,2024-11-30 03:00:00,Набор Mach3 с лосьоном после бритья,613,18,2.380000,11.806667,146.66,0,1,2.380000,1,911,21693077,WB-Smart-Market,Wildberries


In [ ]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,123116122,2024-11-01 03:00:00,Эпилятор женский электрический 3в1 CR-8803,2313,72,120.74,0,0,0,0,20139599,WB-GutenTech,Wildberries,2024-11-13,2024-09-24,2024-10-30,CRONIER_1 SKU_24.09.2024,Кампания завершена,Автоматическая кампания
1,135577637,2024-11-01 03:00:00,Сменные кассеты для бритья Fusion 5 ProGlide 2 шт,2373,68,222.85,0,1,1,1488,19263710,WB-GutenTech,Wildberries,2024-12-09,2024-08-13,2024-11-12,GILLETTE_Fusion5 Proglide + Power_3 SKU_13.08....,Кампания завершена,Автоматическая кампания
2,137609384,2024-11-01 03:00:00,Бритва станок Fusion5 ProGlide Power Flexball ...,1844,72,172.25,0,1,1,2259,19263710,WB-GutenTech,Wildberries,2024-12-09,2024-08-13,2024-11-12,GILLETTE_Fusion5 Proglide + Power_3 SKU_13.08....,Кампания завершена,Автоматическая кампания
3,137609389,2024-11-01 03:00:00,Бритва станок Fusion 5 ProGlide с 3 кассетами,1618,38,152.17,0,1,1,2043,19263710,WB-GutenTech,Wildberries,2024-12-09,2024-08-13,2024-11-12,GILLETTE_Fusion5 Proglide + Power_3 SKU_13.08....,Кампания завершена,Автоматическая кампания
4,149281061,2024-11-01 03:00:00,Кусторез аккумуляторный садовый CSH 360,173,0,16.08,0,0,0,0,20175678,WB-GutenTech,Wildberries,2024-11-18,2024-09-25,2024-09-25,Ножницы_2 SKU_25.09.2024,Кампания завершена,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,288465478,2024-11-30 03:00:00,Набор Breeze бритва с кассетой и шампунь 250мл,204,3,50.51,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
2383,288465480,2024-11-30 03:00:00,Набор Gillette Venus с гелем для бритья,340,10,76.96,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
2384,288465481,2024-11-30 03:00:00,Набор Gillette Venus с чехлом для бритвы,351,15,77.30,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
2385,288465484,2024-11-30 03:00:00,Набор Mach3 с лосьоном после бритья,613,18,146.66,0,1,1,911,21693077,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2025-02-17,Gillette_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')

In [ ]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [ ]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
